Bayu Kurniawan
21/480093/GE/09662

IMPORT LIBRARY

In [1]:
import albumentations as A
import rasterio
import glob
import os
import numpy as np

MENDEFINISIKAN TRANSFORMASI AUGMENTASI YANG AKAN DITERAPKAN PADA DATASET (GAMBAR)

In [2]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(p=0.5)
])

MENDEFINISIKAN LOKASI ATAU DIREKTORI DATASET YANG AKAN DILAKUKAN AUGMENTASI

In [3]:
Image_Drive = "./Data/image3"
Label_Drive = "./Data/label3"

image_paths = glob.glob(os.path.join(Image_Drive, "*.tif"))
label_paths = glob.glob(os.path.join(Label_Drive, "*.tif"))

image_size = len(image_paths)
label_size = len(label_paths)

print(image_size)
print(label_size)

1086
1086


In [4]:
image_list = []
label_list = []

for img_path, label_path in zip(image_paths, label_paths):
    with rasterio.open(img_path) as img_file:
        img_array = img_file.read()
        img_array = np.moveaxis(img_array, 0, 2)  # Move channel to the last dimension
        image_list.append((img_path, img_array))

    with rasterio.open(label_path) as label_file:
        label_array = label_file.read()
        label_array = np.moveaxis(label_array, 0, 2)  # Move channel to the last dimension
        label_list.append((label_path, label_array))

print(len(image_list))
print(len(label_list))

1086
1086


MELAKUKAN TRANSFORMASI AUGMENTASI PADA DATASET (GAMBAR) KEMUDIAN MENYIMPANNYA PADA FOLDER OUTPUT YANG TELAH DITENTUKAN

In [5]:
Output_Image_Drive = "./Data/transformed_images"
Output_Label_Drive = "./Data/transformed_labels"

# Apply the transformations to each image-mask pair individually
for (img_path, img), (label_path, mask) in zip(image_list, label_list):
    transformed = transform(image=img, mask=mask)
    transformed_img = transformed['image']
    transformed_mask = transformed['mask']
    
    # Move channel axis back to the first position for saving with rasterio
    transformed_img = np.moveaxis(transformed_img, 2, 0)
    transformed_mask = np.moveaxis(transformed_mask, 2, 0)
    
    # Save transformed image
    img_output_path = os.path.join(Output_Image_Drive, "augmented_{}".format(os.path.basename(img_path)))
    with rasterio.open(
        img_output_path, 'w',
        driver='GTiff',
        height=transformed_img.shape[1],
        width=transformed_img.shape[2],
        count=transformed_img.shape[0],
        dtype=transformed_img.dtype
    ) as dst:
        dst.write(transformed_img)
    
    # Save transformed mask
    mask_output_path = os.path.join(Output_Label_Drive, "augmented_{}".format(os.path.basename(label_path)))
    with rasterio.open(
        mask_output_path, 'w',
        driver='GTiff',
        height=transformed_mask.shape[1],
        width=transformed_mask.shape[2],
        count=transformed_mask.shape[0],
        dtype=transformed_mask.dtype
    ) as dst:
        dst.write(transformed_mask)

print(f"Transformed images and masks have been saved.")

c:\Users\ASUS\anaconda3\Lib\site-packages\rasterio\__init__.py:327: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


Transformed images and masks have been saved.
